<a href="https://colab.research.google.com/github/ferperezh/IIC3633_ProyectoMetricaDiversidad/blob/main/restaurante/restaurante_metrica_diversidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Métrica Diversidad de Usuario en los Restaurante

Importa los datos ya procesados de los restuarantes primero debes subir el archivo llamado `data_restaurante_user_filtrado.zip` y descomprimirlo. Luego obtenemos la tabla de los usuarios con ratings y con generos.


In [2]:
# Instalar gdown si no está disponible
!pip install gdown

In [20]:
!pip install fastFM
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastFM: filename=fastFM-0.2.10-cp310-cp310-linux_x86_64.whl size=591779 sha256=92f7ba568108985ec09b0bcd59dd7b9f7e87bbb84d9442d1a869a03e51d1a514
  Stored in directory: /root/.cache/pip/wheels/93/92/52/2da7997fcb7a7ce9042ff3b33836ef0c2fd47aa95382d7a113
Successfully built fastFM


In [3]:
# Enlace del archivo en Google Drive
file_url = "https://drive.google.com/uc?id=1RmqpqmpziTegxlBcs1RxsSecdVthV9qU"

In [4]:
# Nombre del archivo descargado
output_name = "data_restaurante_user_filtrado.csv"

In [5]:
# Descargar el archivo
!gdown {file_url} -O {output_name}

print(f"Archivo descargado como: {output_name}")

Downloading...
From (original): https://drive.google.com/uc?id=1RmqpqmpziTegxlBcs1RxsSecdVthV9qU
From (redirected): https://drive.google.com/uc?id=1RmqpqmpziTegxlBcs1RxsSecdVthV9qU&confirm=t&uuid=89605ee4-e865-417c-8578-e7a2bf5021d2
To: /content/data_restaurante_user_filtrado.csv
100% 2.21G/2.21G [00:48<00:00, 46.0MB/s]
Archivo descargado como: data_restaurante_user_filtrado.csv


In [6]:
!ls

data_restaurante_user_filtrado.csv  sample_data


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar el dataset filtrado
data_path = 'data_restaurante_user_filtrado.csv'
data_restaurante_user_filtrado = pd.read_csv(data_path)

In [8]:
data_restaurante_user_filtrado.head()

,user_id,business_id,stars_x,text,date,name,categories,relevant_category,mean_stars
0,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,Zaika,"Halal, Pakistani, Restaurants, Indian",Vegetarian & Specialty Diets,4.8
1,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ...",American Cuisine,3.0
2,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,Dmitri's,"Mediterranean, Restaurants, Seafood, Greek",European Cuisine,1.0
3,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,Fries Rebellion,"Beer Bar, Bars, American (New), Gastropubs, Re...",American Cuisine,5.0
4,9OAtfnWag-ajVxRbUTGIyg,lj-E32x9_FA7GmUrBGBEWg,4.0,Love going here for happy hour or dinner! Gre...,2014-06-27 22:44:01,Brio Italian Grille,"Bars, Pizza, Nightlife, Cocktail Bars, Italian...",European Cuisine,4.0


In [9]:
# Cantidad de filas data_restaurante_user_filtrado
data_restaurante_user_filtrado.shape[0], data_restaurante_user_filtrado['user_id'].nunique()

(3252880, 1437290)

In [10]:
user_ids = data_restaurante_user_filtrado['user_id'].value_counts()


# Ordenar de mayor a menor el count
user_ids = user_ids.sort_values(ascending=False)

user_ids.head()

,count
user_id,
_BcWyKQL16ndpBdggh2kNA,983
1HM81n6n4iPIFU5d2Lokhw,854
ET8n-r7glWYqZhuR6GcdNw,849
fr1Hz2acAb3OaL3l6DyKNg,831
-G7Zkl1wIWBBmD0KRy_sCw,729


In [11]:
# Obtener el 1% de los user_ids
k = int(len(user_ids)* 0.01)
k

14372

In [12]:
user_ids_top = user_ids.head(k)
user_ids_top

,count
user_id,
_BcWyKQL16ndpBdggh2kNA,983
1HM81n6n4iPIFU5d2Lokhw,854
ET8n-r7glWYqZhuR6GcdNw,849
fr1Hz2acAb3OaL3l6DyKNg,831
-G7Zkl1wIWBBmD0KRy_sCw,729
...,...
4PI1QmjBSR30NX5FdUxGKw,20
OqEP5BAqe599Fxra7Xe_bw,20
gkjzvW_tjriBUV3_bvOs2w,20


In [13]:
# filtramos por solo estos users_ids_top
data_rest_sample = data_restaurante_user_filtrado[data_restaurante_user_filtrado['user_id'].isin(user_ids_top.index)]
data_rest_sample = data_rest_sample[['user_id', 'business_id', 'name', 'relevant_category']]
data_rest_sample.head()

,user_id,business_id,name,relevant_category
1,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,Melt,American Cuisine
5,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,LaScala's,European Cuisine
8,j2wlzrntrbKwyOcOiB3l3w,rBdG_23USc7DletfZ11xGA,Olio,European Cuisine
9,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,Good Karma Cafe,Beverages & Desserts
10,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,Naked Tchopstix Express,Asian Cuisine


In [14]:
data_rest_sample.shape[0], data_rest_sample['user_id'].nunique()

(634525, 14372)

In [15]:
from sklearn.model_selection import train_test_split

data_rest_train, data_rest_test = train_test_split(
    data_rest_sample,
    test_size=0.2,
    stratify=data_rest_sample['user_id'],  # Stratificar por `user_id`
    random_state=42
)

# Generar Recomendaciones

Se probará con diferentes métodos de recomendación para evaluar los resultados del modelo:


## Método Most Popular:
Recomienda los restaurante más escuchadas en el conjunto global de datos de entrenamiento.
- Cuenta y ordena los restaurante según número de apariciones
- Selecciona los `k restaurantes` más populares
- Alta probabilidad de recomendar contenido con hartas visualizaciones
- Mismo resultado para todos los usuarios
- Baja diversidad de recomendaciones

### Procesamiento de Datos para Most Popular
Usar el metodo Most popular tomaremos las canciones descargadas/escuhadas por cada usuario como las canciones que quieren cada usaurio. Por lo que agregamos una columna llamada `Popularity` a la tabla de canciones que seria la cantidad de usaurios o veces que se descarga una cancion. Para luego recomendar canciones a cada usuario.

In [16]:
def get_top_k_categories(df, k):
    # Crear un subconjunto con los usuarios y categorías relevantes
    user_category_df = df[['user_id', 'relevant_category']]

    # Calcular las k categorías más frecuentes por cada usuario
    top_k_categories = (
        user_category_df.groupby(['user_id', 'relevant_category'])
        .size()  # Contar ocurrencias de cada categoría por usuario
        .reset_index(name='count')  # Resetear índice y nombrar la columna de conteo
        .sort_values(['user_id', 'count'], ascending=[True, False])  # Ordenar por usuario y conteo descendente
        .groupby('user_id')  # Agrupar por usuario
        .head(k)  # Tomar las K categorías más frecuentes por usuario
    )

    return top_k_categories


### Recomendación de Most Popular

Recomienda los restaurantes más escuchadas en el conjunto global de datos de entrenamiento.
- Cuenta y ordena los restaurantes según número de reproducciones
- Selecciona los `k restaurantes` más populares
- Alta probabilidad de recomendar contenido con hartas visualizaciones
- Mismo resultado para todos los usuarios
- Baja diversidad de recomendaciones

In [17]:
# Función para recomendar los restaurantes más populares
def recommend_most_popular(user, data_train, k=10):
    # Agrupar por business_id contando apariciones
    data_business_count = data_train.groupby('business_id')['business_id'].count().reset_index(name='popularity')
    # Mapear nombres de restaurantes
    data_business_count['name'] = data_business_count['business_id'].map(dict(zip(data_train['business_id'], data_train['name'])))
    # Ordenar por popularidad y obtener los k más populares
    data_business_count = data_business_count.sort_values('popularity', ascending=False)
    top_k_popular_business = data_business_count.head(k)
    return top_k_popular_business['name'].tolist()

## Método Random
Recomienda restaurantes de manera aleatoria del conjunto de entrenamiento.
- Selecciona `k` restaurantes aleatorias
- Usado como baseline para comparaciones
- Debiese mostrar un aumento en la diversidad de las recomendaciones

In [18]:
import random
# Función para recomendar restaurantes aleatorios
def recommend_random(user, data_train, k=10):
    # Obtener restaurantes únicos
    unique_businesses = data_train['business_id'].unique()
    # Seleccionar k aleatorios
    random_businesses = random.sample(list(unique_businesses), min(k, len(unique_businesses)))
    # Obtener nombres de restaurantes
    random_business_names = [data_train[data_train['business_id'] == business_id]['name'].iloc[0] for business_id in random_businesses]
    return random_business_names

## Método FastFM
FastFM es un modelo basado en Factorization Machines, utilizado para generar recomendaciones. Este enfoque:

- Aprende interacciones latentes entre usuarios y canciones a partir de datos dispersos.
- Es especialmente eficaz para integrar tanto características del usuario como del contenido, además de las interacciones implícitas.
- Emplea técnicas de factorización matricial y optimización eficiente para manejar grandes conjuntos de datos.

In [22]:
from fastFM import sgd
from scipy import sparse


def recommend_fastfm(user, data_train, k=10):
# COLUMNAS: Username,	artist_song, 	genre, 	user_id, 	artist_song_id
  # Creamos matriz de interacción
  us_matrix = pd.pivot_table(
    data_train,
    index='user_id',
    columns='artist_song',
    values='artist_song_id',
    aggfunc='count',
    fill_value=0
  )
  # Expandir matriz en formato largo
  interaction_data = us_matrix.stack().reset_index()
  interaction_data.columns = ['user_id', 'artist_song', 'interaction']
  interaction_data['interaction'] = (interaction_data['interaction'] > 0).astype(int)  # Binarización

  # Crear matriz dispersa para características
  X = sparse.csc_matrix(pd.get_dummies(interaction_data[['user_id', 'artist_song']], sparse=True).values)

  # Etiquetas (1 si hubo interacción, 0 si no)
  y = interaction_data['interaction'].values

  # Creamos el modelo de FastFM fit con primera columna de objetivo.
  ffm = sgd.FMRegression(n_iter=100, rank=2)
  ffm.fit(X, y)

  # Obtener vector del usuario para predicciones
  user_songs = us_matrix.loc[user]
  unseen_songs = user_songs[user_songs == 0].index  # Canciones no escuchadas
  user_data = pd.DataFrame({'user_id': [user] * len(unseen_songs), 'artist_song': unseen_songs})

  # Matriz dispersa para predicciones
  feature_columns = pd.get_dummies(interaction_data[['user_id', 'artist_song']], sparse=True).columns
  X_user = sparse.csc_matrix(pd.get_dummies(user_data, sparse=True).reindex(columns=feature_columns).fillna(0).values)
  predicted_ratings = ffm.predict(X_user)

  # Ordenar y seleccionar las mejores recomendaciones
  user_data['predicted_rating'] = predicted_ratings
  top_recommendations = user_data.sort_values('predicted_rating', ascending=False).head(k)['artist_song'].tolist()

  return top_recommendations


## Método DeepFM
DeepFM es un modelo avanzado para sistemas de recomendación que combina lo mejor de las Factorization Machines (FMs) y las redes neuronales profundas (Deep Learning) para capturar interacciones complejas entre características. Este enfoque:

- Factorization Machines (FMs): Capturan de manera eficiente las interacciones de segundo orden entre características, como usuario y canción, especialmente en datos dispersos.
- Redes neuronales profundas: Aprenden interacciones de mayor nivel (no lineales) entre características, mejorando la capacidad del modelo para capturar patrones complejos.
- Integra ambos componentes en un solo modelo, compartiendo los mismos embeddings para reducir la redundancia y optimizar el aprendizaje.

In [23]:
from tensorflow import keras
import tensorflow as tf
import random

# Creamos el modelo de DeepFM
def build_deepfm(input_dim):
    # Componente FM
    input_movie = keras.layers.Input(shape=(input_dim,))
    fm_linear = keras.layers.Dense(1)(input_movie)

    # Componente Deep
    deep = keras.layers.Dense(256, activation='relu')(input_movie)
    deep = keras.layers.Dropout(0.2)(deep)
    deep = keras.layers.Dense(128, activation='relu')(deep)
    deep = keras.layers.Dense(64, activation='relu')(deep)
    deep = keras.layers.Dense(1)(deep)

    # Combinar Componentes
    fm_deep = keras.layers.Concatenate()([fm_linear, deep])

    # Combinar Componentes
    output = keras.layers.Add()([fm_linear, deep])

    # Modelo
    model = keras.Model(inputs=input_movie, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

def recommend_deepfm(user, data_train, k=10):
  # Generamos subset de datos para evaluar interacción usuario-canción
  # COLUMNAS: userId, movieId, title, relevant_relevant_genre
  user_movie = data_train[['userId', 'title', 'relevant_genre']]

  # Creamos matriz de interacción
  us_matrix = pd.pivot_table(
    user_movie,
    values='relevant_genre',           # Género es el valor de interacción
    index='userId',
    columns='title',
    aggfunc='count',
    fill_value=0
  )

  # Preparar datos para DeepFM
  X = us_matrix.values
  input_dim = X.shape[1]
  y = us_matrix.iloc[:,0].values

  # Entrenamos el modelo
  model = build_deepfm(input_dim)
  model.fit(X, y, epochs=5, batch_size=32, verbose=0)

  # Calcular las recomendaciones para el usuario
  user_vector = us_matrix.loc[user].values.reshape(1, -1)
  predictions = model.predict(user_vector).flatten()

  # Seleccionar las Top-K canciones
  movie_ids = us_matrix.columns
  already_heard = set(user_movie[user_movie['userId'] == user]['title'])

  # Filtrar y ordenar las recomendaciones
  candidate_movie = [(movie, score) for movie, score in zip(movie_ids, predictions)
                    if movie not in already_heard]
  # Si no encuentra candidatos, entones se usan las recomendaciones originales
  if not candidate_movie:
      candidate_movie = [(movie, score) for movie, score in zip(movie_ids, predictions)]

  candidate_movie.sort(key=lambda x: x[1], reverse=True)

  # Asegurarse de que hayan k recomendaciones
  if len(candidate_movie) < k:
      # Rellenar con canciones aleatorias si es necesario
      available_movie = list(set(movie_ids) - set(movie for movie, _ in candidate_movie))
      if available_movie:
        padding = random.sample(available_movie, min(k - len(candidate_movie), len(available_movie)))
        candidate_movie.extend([(movie, 0.0) for movie in padding])

  # Retornar las k recomendaciones
  recommendations = [movie for movie, _ in candidate_movie[:k]]

  return recommendations


# Calculamos Métricas:

## Metricas de Precisión
- MAP
- NDCG@5
- Precision@10
- Recall@10

In [24]:
def mean_average_precision(y_true, y_pred, k):
    aps = []
    for true, pred in zip(y_true, y_pred):
        relevant = 0
        precision_sum = 0
        for i in range(min(k, len(pred))):
            if pred[i] in true:
                relevant += 1
                precision_sum += relevant / (i + 1)
        aps.append(precision_sum / len(true) if true else 0)
    return sum(aps) / len(aps)


In [25]:
def ndcg_at_k(y_true, y_pred, k):
    ndcgs = []
    for true, pred in zip(y_true, y_pred):
        dcg = sum([int(pred[i] in true) / np.log2(i + 2) for i in range(min(k, len(pred)))])
        idcg = sum([1 / np.log2(i + 2) for i in range(min(k, len(true)))])
        ndcgs.append(dcg / idcg if idcg > 0 else 0)
    return sum(ndcgs) / len(ndcgs)


In [26]:
def precision_at_k(y_true, y_pred, k):
    precisions = []
    for true, pred in zip(y_true, y_pred):
        relevant = sum([1 for i in pred[:k] if i in true])
        precisions.append(relevant / k)
    return sum(precisions) / len(precisions)

def recall_at_k(y_true, y_pred, k):
    recalls = []
    for true, pred in zip(y_true, y_pred):
        relevant = sum([1 for i in pred[:k] if i in true])
        recalls.append(relevant / len(true) if true else 0)
    return sum(recalls) / len(recalls)


## Métricas de Diversidad: ¡Enfoque del Estudio!
- **User Diversity**: Métrica propuesta por la investigación
- Long Tail
- Shannon Entropy
- Intra List Diversity
- Diversity Coverage
- Inverse Propensity Score

### Definición de **USER DIVERSITY**, la métrica de la investigación.
1. `UD = 1 - (|∑{j=1...k}[(R_j/R)*log(R_j/R)]| / log(k))`
2. `k`: número de categorías para un usuario.
3. `R`: número total de recomendaciones.
4. `(R_j/R)`: Proporción de recomendaciones del usuario que pertenecen a la categoría `j`
5. `log(...)`: para penalizar concentración excesiva en una sola categoría.
6. `/ log(k)`: normaliza el valor.
7. `1 - `: Valor resultante entre 0 y 1.
8. *Valor alto*: alta diversidad en las recomendaciones recibidas por el usuario (pertenecen a varias categorías favoritas)

In [27]:
def user_diversity(recommendations, category_map, k):
    user_diversities = []
    for user_recommendations in recommendations:
        category_counts = {}
        for item in user_recommendations[:k]:
            category = category_map.get(item, None)
            if category:
                category_counts[category] = category_counts.get(category, 0) + 1

        R = sum(category_counts.values())
        UD = 0
        for Rj in category_counts.values():
            if Rj > 0:
                UD += (Rj / R) * np.log(Rj / R)
        UD = -UD / np.log(k) if k > 1 else 0
        user_diversities.append(1 - UD)
    return sum(user_diversities) / len(user_diversities)


### **Long Tail**:
Mide que tan diversas son las recomendaciones en términos de popularidad de los items.
1.  Equación: `LT = |Intersection(Rec, TailItems)| / |Rec|`
2. Parametro `beta=0.4`: define umbral para considerar item como parte de la cola larga.
3. *Valor alto*: se recomiendan más items poco populares



In [28]:
def long_tail(recommendations, popularity_map, threshold):
    long_tail_items = [item for item, pop in popularity_map.items() if pop <= threshold]
    long_tail_recommendations = sum([item in long_tail_items for recs in recommendations for item in recs])
    total_recommendations = sum([len(recs) for recs in recommendations])
    return long_tail_recommendations / total_recommendations


### **Entropía de Shannon**:
Mide la incertidumbre o aleatoriedad de la distribución de recomendaciones.
1. `H = -∑(pi*log2(pi)) / log2(n)`
2. `pi`: probabilidad de cada item en las recomendaciones
3. *Valor alto*: mayor aleatoriedad/diversidad en las recomendaciones.


In [29]:
def shannon_entropy(recommendations, category_map):
    entropies = []
    for user_recommendations in recommendations:
        category_counts = {}
        for item in user_recommendations:
            category = category_map.get(item, None)
            if category:
                category_counts[category] = category_counts.get(category, 0) + 1
        total = sum(category_counts.values())
        entropy = -sum([(count / total) * np.log2(count / total) for count in category_counts.values() if count > 0])
        entropies.append(entropy)
    return sum(entropies) / len(entropies)


### **Intra List Diversity**:
Calcula diversidad basándose en la similitud entre los items.
1. `ILD = ∑∑(d(i,j)) / (n*(n-1)/2)`
2. `d(i,j)`: es la distancia entre los items `i` y `j`, usando coseno
3. *Valor alto*: mayor diversidad entre los items recomendados.


In [30]:
def intra_list_diversity(recommendations, category_map):
    ild_scores = []
    for user_recommendations in recommendations:
        categories = [category_map.get(item, None) for item in user_recommendations]
        unique_categories = set(categories)
        ild = len(unique_categories) / len(categories) if categories else 0
        ild_scores.append(ild)
    return sum(ild_scores) / len(ild_scores)


### **Diversity Coverage**:
Mide la cobertura como la proporción de items únicos recomendados respceto al total de items posibles.
1. `DC = |Unique_Rec| / |All Items|`
2. *Valor alto*: se están recomendando items de todo el catálogo

In [31]:
def diversity_coverage(recommendations, category_map):
    all_categories = set(category_map.values())
    recommended_categories = set(category_map[item] for recs in recommendations for item in recs if item in category_map)
    return len(recommended_categories) / len(all_categories)


### **Inverse Propensity Score** (IPS):
Penaliza la recomendación de items muy populares
1. `IPS = (1/p(i))`
2. Parametro `lambda` para suavizar la propensidad
3. `p(i)`: probabilidad que el item `i` sea seleccionado
4. *Valor alto*: se están recomendando más items poco probables.

In [32]:
def inverse_propensity_score(recommendations, propensity_map):
    ips_scores = []
    for user_recommendations in recommendations:
        scores = [1 / propensity_map[item] for item in user_recommendations if item in propensity_map]
        ips_scores.append(sum(scores) / len(scores) if scores else 0)
    return sum(ips_scores) / len(ips_scores)


# Ejecución de Métodos

Creamos una función para ejecutar el modelo con parametros modificables correspondientes a:
- `test_data`: Datos para testeo del modelo
- `train_data`: Datos para entrenamiento del modelo
- `top_n`: Para recommendación *Modelo* (Most Popular, Random, Collaborative, Hibrido)
- `k`: Cuantos clusters de géneros se considerarán para los usuarios
- `beta_lt`: Parametro beta de métrica *Long Tail*, define umbral para considerar un item como parte de la *Long Tail*
- `lambda_ips`: Parámetro de métrica *Inverse Propensity Score*, sirve para suavizar el resultado.

In [33]:
def most_popular_metrics(test_data, train_data, top_n, k, beta_lt=0.4, lambda_ips=0.5):
    map_sum = 0
    ndcg_sum = 0
    precision_sum = 0
    recall_sum = 0
    diversity_sum = 0
    lt_sum = 0
    entropy_sum = 0
    ild_sum = 0
    dc_sum = 0
    ips_sum = 0
    user_count = 0

    # Obtener las categorías principales de prueba
    top_k_categories_data = get_top_k_categories(test_data, k)

    # Crear mapa de categorías (business_id -> relevant_category)
    category_map = dict(zip(train_data['business_id'], train_data['relevant_category']))

    for userid in test_data['user_id'].unique():
        # Generar la lista de recomendaciones para el usuario actual
        recommend_list = recommend_most_popular(userid, train_data, top_n)

        # Calcular métricas para el usuario
        map_user = mean_average_precision([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        ndcg_user = ndcg_at_k([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        precision_user = precision_at_k([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        recall_user = recall_at_k([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        diversity_user = user_diversity([recommend_list], category_map, k)
        lt_user = long_tail([recommend_list], dict(train_data['business_id'].value_counts()), beta_lt)
        entropy_user = shannon_entropy([recommend_list], category_map)
        ild_user = intra_list_diversity([recommend_list], category_map)
        dc_user = diversity_coverage([recommend_list], category_map)
        ips_user = inverse_propensity_score([recommend_list], dict(train_data['business_id'].value_counts(normalize=True)))

        # Sumar las métricas
        map_sum += map_user
        ndcg_sum += ndcg_user
        precision_sum += precision_user
        recall_sum += recall_user
        diversity_sum += diversity_user
        lt_sum += lt_user
        entropy_sum += entropy_user
        ild_sum += ild_user
        dc_sum += dc_user
        ips_sum += ips_user
        user_count += 1

    # Promedio de métricas
    return {
        "MAP": map_sum / user_count,
        "nDCG": ndcg_sum / user_count,
        "Precision": precision_sum / user_count,
        "Recall": recall_sum / user_count,
        "User_Diversity": diversity_sum / user_count,
        "Long_Tail": lt_sum / user_count,
        "Shannon_Entropy": entropy_sum / user_count,
        "Intra_List_Diversity": ild_sum / user_count,
        "Diversity_Coverage": dc_sum / user_count,
        "Inverse_Propensity_Score": ips_sum / user_count
    }


## Metodo Random

In [34]:
def random_metrics(test_data, train_data, top_n, k, beta_lt=0.4, lambda_ips=0.5):
    map_sum = 0
    ndcg_sum = 0
    precision_sum = 0
    recall_sum = 0
    diversity_sum = 0
    lt_sum = 0
    entropy_sum = 0
    ild_sum = 0
    dc_sum = 0
    ips_sum = 0
    user_count = 0

    # Obtener las categorías principales de prueba
    top_k_categories_data = get_top_k_categories(test_data, k)

    # Crear mapa de categorías (business_id -> relevant_category)
    category_map = dict(zip(train_data['business_id'], train_data['relevant_category']))

    for userid in test_data['user_id'].unique():
        # Generar la lista de recomendaciones para el usuario actual
        recommend_list = recommend_random(userid, train_data, top_n)

        # Calcular métricas para el usuario
        map_user = mean_average_precision([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        ndcg_user = ndcg_at_k([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        precision_user = precision_at_k([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        recall_user = recall_at_k([test_data[test_data['user_id'] == userid]['business_id'].tolist()], [recommend_list], top_n)
        diversity_user = user_diversity([recommend_list], category_map, k)
        lt_user = long_tail([recommend_list], dict(train_data['business_id'].value_counts()), beta_lt)
        entropy_user = shannon_entropy([recommend_list], category_map)
        ild_user = intra_list_diversity([recommend_list], category_map)
        dc_user = diversity_coverage([recommend_list], category_map)
        ips_user = inverse_propensity_score([recommend_list], dict(train_data['business_id'].value_counts(normalize=True)))

        # Sumar las métricas
        map_sum += map_user
        ndcg_sum += ndcg_user
        precision_sum += precision_user
        recall_sum += recall_user
        diversity_sum += diversity_user
        lt_sum += lt_user
        entropy_sum += entropy_user
        ild_sum += ild_user
        dc_sum += dc_user
        ips_sum += ips_user
        user_count += 1

    # Promedio de métricas
    return {
        "MAP": map_sum / user_count,
        "nDCG": ndcg_sum / user_count,
        "Precision": precision_sum / user_count,
        "Recall": recall_sum / user_count,
        "User_Diversity": diversity_sum / user_count,
        "Long_Tail": lt_sum / user_count,
        "Shannon_Entropy": entropy_sum / user_count,
        "Intra_List_Diversity": ild_sum / user_count,
        "Diversity_Coverage": dc_sum / user_count,
        "Inverse_Propensity_Score": ips_sum / user_count
    }


## Ejecución y obtención de métricas

Ahora con las funciones definidas para obtener las metricas las ejecutamos para obtener las metrcicas con 10 recomendaciones por modelo, 5 clusters por usuario, es decir 3 top generos por usuario. La beta para el long tail es de 40% mientras que para ips es de 50%.

### Método Most Popular

In [ ]:
metric_results = most_popular_metrics(data_rest_test, data_rest_train, 10, 3, beta_lt=0.4, lambda_ips=0.5)
print(f"MAP: {metric_results['MAP']}")
print(f"nDCG: {metric_results['nDCG']}")
print(f"Precision: {metric_results['Precision']}")
print(f"Recall: {metric_results['Recall']}")
print(f"User_Diversity: {metric_results['User_Diversity']}")
print(f"Long_Tail: {metric_results['Long_Tail']}")
print(f"Shannon_Entropy: {metric_results['Shannon_Entropy']}")
print(f"Intra_List_Diversity: {metric_results['Intra_List_Diversity']}")
print(f"Diversity_Coverage: {metric_results['Diversity_Coverage']}")
print(f"Inverse_Propensity_Score: {metric_results['Inverse_Propensity_Score']}")

## Metodo Random

In [ ]:
metric_results_random = random_metrics(data_rest_test, data_rest_train, 10, 3, beta_lt=0.4, lambda_ips=0.5)
print(f"MAP: {metric_results_random['MAP']}")
print(f"nDCG: {metric_results_random['nDCG']}")
print(f"Precision: {metric_results_random['Precision']}")
print(f"Recall: {metric_results_random['Recall']}")
print(f"User_Diversity: {metric_results_random['User_Diversity']}")
print(f"Long_Tail: {metric_results_random['Long_Tail']}")
print(f"Shannon_Entropy: {metric_results_random['Shannon_Entropy']}")
print(f"Intra_List_Diversity: {metric_results_random['Intra_List_Diversity']}")
print(f"Diversity_Coverage: {metric_results_random['Diversity_Coverage']}")
print(f"Inverse_Propensity_Score: {metric_results_random['Inverse_Propensity_Score']}")